# House Data Lookup using Zillow API

> In this project I am building upon the authors work. This will give estimated profit for a potential buy to purchase the home as a rental property

Click [HERE](https://www.youtube.com/watch?v=o8Dfa3hUHtc) and [HERE](https://github.com/analyticsariel/projects/blob/master/property_data/zillow_property_data_api.ipynb) to see the references and templates I used for this work.

### Import Packages

In [1]:
from googlesearch import search
import pandas as pd
import requests
import json
import time
import io
import plotly.express as px

In [2]:
# Set up Notebook

# show all columns
pd.set_option('display.max_columns', None)

## Run the Google Search for the URL of a property

In [6]:
# Create variable for search criteria / Address
property_address = "6520 Roaring Creek, Arygle, TX 76226"
query = ' zillow homedetails' + property_address

In [7]:
# Create a list of the top 3 search results

results = search(query, num_results=3 )
results_list = [u for u in results] 
results_list


['https://www.zillow.com/homedetails/6520-Roaring-Crk-Argyle-TX-76226/243579919_zpid/',
 'https://www.realtor.com/realestateandhomes-detail/6520-Roaring-Crk_Argyle_TX_76226_M85977-23208',
 'https://www.zillow.com/homedetails/6521-Roaring-Crk-Argyle-TX-76226/243578973_zpid/']

In [17]:
# Since "Zillow Home Details" is in the query it should be the number one result
url = results_list[0]
print(url)

https://www.zillow.com/homedetails/6520-Roaring-Crk-Argyle-TX-76226/243579919_zpid/


> The ZPID is what the Zillow API uses to find the data. The ZPID is the numeric part of the url ending with _zpid/

In [18]:
# Get the ZPID to search the property using the API
zpid = [x for x in url.split('/') if 'zpid' in x][0].split('_')[0]
print('Zpid of the property is:', zpid)


Zpid of the property is: 243579919


# Use API to get property details

In [11]:
# This code comes directly from rapidapi.com

url = "https://zillow-com1.p.rapidapi.com/property"

querystring = {"zpid":zpid}

headers = {
"X-RapidAPI-Key": "8246fc7aefmshce6cca477ba63adp1661d4jsn1f03c003e6b2",
"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

response.status_code
# 200 indicates successful request

200

In [12]:
# Here is all of the data from the address listed

response.json()

{'listingProvider': None,
 'zpid': 243579919,
 'buildingPermits': None,
 'propertyTaxRate': 1.77,
 'contact_recipients': [{'agent_reason': 1,
   'zpro': None,
   'recent_sales': 0,
   'review_count': 2,
   'display_name': 'Timothy Williams',
   'zuid': 'X1-ZUws4qoe0h7nrd_282em',
   'rating_average': 5,
   'badge_type': 'Premier Agent',
   'phone': {'prefix': '206', 'areacode': '903', 'number': '2778'},
   'image_url': 'https://photos.zillowstatic.com/h_n/ISn24r86hdyzse0000000000.jpg'}],
 'zipcode': '76226',
 'openHouseSchedule': {},
 'longitude': -97.199524,
 'zestimateLowPercent': '6',
 'address': {'community': None,
  'city': 'Argyle',
  'state': 'TX',
  'neighborhood': None,
  'subdivision': None,
  'streetAddress': '6520 Roaring Crk',
  'zipcode': '76226'},
 'cityId': 4351,
 'timeOnZillow': '79 days',
 'url': '/homedetails/6520-Roaring-Crk-Argyle-TX-76226/243579919_zpid/',
 'zestimate': 407000,
 'imgSrc': 'https://photos.zillowstatic.com/fp/a9191e0d1840e882db003d765de42fa6-p_d.jpg'

In [13]:
# transform data to pandas dataframe
df_property_detail = pd.json_normalize(data=response.json())
print('Num of rows:', len(df_property_detail))
print('Num of cols:', len(df_property_detail.columns))
df_property_detail.head()

Num of rows: 1
Num of cols: 283


,listingProvider,zpid,buildingPermits,propertyTaxRate,contact_recipients,zipcode,longitude,zestimateLowPercent,cityId,timeOnZillow,url,zestimate,imgSrc,description,price,livingAreaValue,taxHistory,brokerId,stateId,streetAddress,solarPotential,dateSold,countyId,timeZone,homeType,livingAreaUnits,comingSoonOnMarketDate,rentZestimate,bathrooms,annualHomeownersInsurance,state,propertyTypeDimension,building,yearBuilt,brokerageName,pageViewCount,county,mortgageRates,monthlyHoaFee,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,livingArea,priceHistory,favoriteCount,schools,zestimateHighPercent,mlsid,countyFIPS,city,providerListingID,country,currency,isListedByOwner,contingentListingType,address.community,address.city,address.state,address.neighborhood,address.subdivision,address.streetAddress,address.zipcode,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.flooring,resoFacts.builderModel,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.totalActualRent,resoFacts.offerReviewDate,resoFacts.horseYN,resoFacts.buyerAgencyCompensationType,resoFacts.belowGradeFinishedArea,resoFacts.feesAndDues,resoFacts.cityRegion,resoFacts.mainLevelBathrooms,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.carportParkingCapacity,resoFacts.inclusions,resoFacts.hasFireplace,resoFacts.gas,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.roadSurfaceType,resoFacts.subdivisionName,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.bathroomsOneQuarter,resoFacts.lotSize,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.bathrooms,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherParking,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenIndoorAirQuality,resoFacts.greenSustainability,resoFacts.heating,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.hasAttachedGarage,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.constructionMaterials,resoFacts.fireplaceFeatures,resoFacts.hoaFeeTotal,resoFacts.doorFeatures,resoFacts.bathroomsPartial,resoFacts.bathroomsHalf,resoFacts.listAOR,resoFacts.buildingName,resoFacts.attic,resoFacts.mainLevelBedrooms,resoFacts.elementarySchool,resoFacts.virtualTour,resoFacts.hasCarport,resoFacts.canRaiseHorses,resoFacts.hasLandLease,resoFacts.communityFeatures,resoFacts.yearBuiltEffective,resoFacts.middleOrJuniorSchool,resoFacts.lotSizeDimensions,resoFacts.hasHomeWarranty,resoFacts.entryLocation,resoFacts.yearBuilt,resoFacts.propertySubType,resoFacts.propertyCondition,resoFacts.utilities,resoFacts.parcelNumber,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.isNewConstruction,resoFacts.otherStructures,resoFacts.livingArea,resoFacts.livingAreaRangeUnits,resoFacts.buildingArea,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.isSeniorCommunity,resoFacts.foundationDetails,resoFacts.frontageLength,resoFacts.associationAmenities,resoFacts.buildingFeatures,resoFacts.hasCooling,resoFacts.buildingAreaSource,resoFacts.parkingFeatures,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.bathroomsFull,resoFacts.additionalParcelsDescription,resoFacts.waterBodyName,resoFacts.waterfrontFeatures,resoFacts.aboveGradeFinishedArea,resoFacts.zoningDescription,resoFacts.levels,resoFacts.basementYN,resoFacts.hasView,resoFacts.appliances,resoFacts.securityFeatures,resoFacts.elevation,resoFacts.fencing,resoFacts.greenEnergyEfficient,resoFacts.developmentStatus,res

In [14]:
taxHistory = df_property_detail['taxHistory'].iloc[0]
print('  taxHistory: {}'.format( taxHistory))

  taxHistory: [{'time': 1658351523018, 'valueIncreaseRate': 0.14716034, 'taxIncreaseRate': 0.12694123, 'taxPaid': 7803.91, 'value': 367619}, {'time': 1626815523018, 'valueIncreaseRate': 0.09724405, 'taxIncreaseRate': 0.066580586, 'taxPaid': 6924.86, 'value': 320460}, {'time': 1595279523018, 'valueIncreaseRate': -0.057466775, 'taxIncreaseRate': -0.08331978, 'taxPaid': 6492.58, 'value': 292059}, {'time': 1563657123018, 'valueIncreaseRate': 8.217539, 'taxIncreaseRate': 7.820311, 'taxPaid': 7082.71, 'value': 309866}, {'time': 1532121123018, 'valueIncreaseRate': 0.16301678, 'taxIncreaseRate': 0, 'taxPaid': 803, 'value': 33617}, {'time': 1500585123018, 'valueIncreaseRate': 0, 'taxIncreaseRate': 0, 'taxPaid': None, 'value': 28905}]


In [15]:
taxHistory = df_property_detail['taxHistory'].iloc[0]

# Sort the tax history by 'time' in descending order to get the most recent entry
sorted_taxHistory = sorted(taxHistory, key=lambda x: x['time'], reverse=True)

# Get the most recent taxPaid value
most_recent_taxPaid = sorted_taxHistory[0]['taxPaid']

print('Most recent taxPaid: {}'.format(most_recent_taxPaid))


Most recent taxPaid: 7803.91


In [21]:
#               Property Information

# school lot_size = df_property_detail['resoFacts.lotSize'].iloc[0]
bedrooms = df_property_detail['bedrooms'].iloc[0]
bathrooms = df_property_detail['bathrooms'].iloc[0]
year_built = df_property_detail['yearBuilt'].iloc[0]
property_type = df_property_detail['homeType'].iloc[0]
living_area = df_property_detail['resoFacts.livingArea'].iloc[0]
lot_dimensions = df_property_detail['resoFacts.lotSizeDimensions'].iloc[0]
zoning = df_property_detail['resoFacts.zoning'].iloc[0]


rent_zestimate = df_property_detail['rentZestimate'].iloc[0]





Most recent taxPaid: 7803.91
PROPERTY DETAILS FOR:  6520 Roaring Creek, Arygle, TX 76226
------------------------------------------------------------------
Price: 407000
Most recent taxPaid: 7803.91
  Bedrooms: 4
  Bathrooms: 3
  Year Built: 2018
  Living Area: 2,222 sqft
  Lot Dimensions: None
  Zoning: None
  Property Type: SINGLE_FAMILY
  Zestimate: $407,000
  Rent Zestimate: $3,024


In [ ]:
################### estimated costs  ###################

# Price:
zestimate = df_property_detail['zestimate'].iloc[0]
down_payment = zestimate * .4


# Payment
principal = zestimate - downpayment
interest_rate = 0.05  # Annual interest rate
loan_term = 30  # Loan term in years

monthly_interest_rate = interest_rate / 12  # Monthly interest rate
num_payments = loan_term * 12  # Total number of payments

# Calculate the monthly mortgage payment
monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -num_payments)

# Above Calculates the estimated monthly payment. 

# Now the rent zestimate 



########            Taxes:  #############


taxHistory = df_property_detail['taxHistory'].iloc[0]

# Sort the tax history by 'time' in descending order to get the most recent entry
sorted_taxHistory = sorted(taxHistory, key=lambda x: x['time'], reverse=True)

# Get the most recent taxPaid value
most_recent_taxPaid = sorted_taxHistory[0]['taxPaid']

print('Most recent taxPaid: {}'.format(most_recent_taxPaid))


# property_tax_rate = df_property_detail['propertyTaxRate'].iloc[0]


In [23]:
print('PROPERTY DETAILS FOR: ', property_address)
print('------------------------------------------------------------------')


print('Price: {}'.format(zestimate))
print('40% Downpayment: {}'.format(down_payment))
print('Based on todays average interest rate of'): {}.format(interest_rate)
print('The payment will be')+(monthly_payment)

print('Most recent taxPaid: {}'.format(most_recent_taxPaid))


print('Monthly mortgage payment: {:.2f}'.format(monthly_payment))



print('  Bedrooms: {}'.format( bedrooms))
print('  Bathrooms: {}'.format( bathrooms))
print('  Year Built: {}'.format( year_built))
print('  Living Area: {}'.format( living_area))
# print('  Lot Size: {}'.format( lot_size))
print('  Lot Dimensions: {}'.format( lot_dimensions))
print('  Zoning: {}'.format( zoning))
print('  Property Type: {}'.format( property_type))
print('  Zestimate: ${:,.0f}'.format( zestimate))
print('  Rent Zestimate: ${:,.0f}'.format( rent_zestimate))





SyntaxError: illegal target for annotation (2228706278.py, line 7)